<a href="https://colab.research.google.com/github/LingZ1993/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

In [ ]:
#filter the dataset for total_votes being greater than or equal to 20
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vote_count = df.filter("total_votes >= 20")
vote_count.show()

In [ ]:
#create a new table to retrieve all the rows where the number of helpful votes divided by total_votes is equal to or greater than 50%
vine_table = vote_count.filter(vote_count["helpful_votes"]/vote_count["total_votes"]>=0.5)
vine_table.show()

In [ ]:
#create a new DataFrame or table that retrieves all the rows where a review was written as part of the vine program (paid), vine == 'Y'
vine_table.filter(vine_table["vine"] == "Y").count()

In [ ]:
#create a new DataFrame or table that retrieves all the rows where a review was written as part of the vine program (paid), vine == 'Y'
vine_table.filter(vine_table["vine"] == "Y").show(10)

In [ ]:
#Create a new table that retrieves all the rows where the review was not written as part of the vine program (unpaid), vine == 'N'
vine_table.filter(vine_table["vine"] == 'N').count()

In [ ]:
#Create a new table that retrieves all the rows where the review was not written as part of the vine program (unpaid), vine == 'N'
vine_table.filter(vine_table["vine"] == 'N').show(10)

In [ ]:
#Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid)
five_star = vine_table.filter(vine_table["star_rating"] == 5)
five_star.cache()

In [ ]:
#... number of five star reviews for both paid and unpaid
five_star.count()

In [ ]:
#... total number of reviews
vine_table.count()

In [ ]:
#... percentage of five star reviews
five_star.count() / vine_table.count()

In [ ]:
#... percentage of unpaid reviews
five_star.filter(five_star["verified_purchase"]=='N').count()/vine_table.filter(vine_table["verified_purchase"]=='N').count()

In [ ]:
#... percentage of paid reviews
five_star.filter(five_star["verified_purchase"]=='Y').count()/vine_table.filter(vine_table["verified_purchase"]=='Y').count()